In [ ]:
# Importing Dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Import the data
data = pd.read_csv('Combined Customers Data - Sheet1.csv')

# Separate the labels (fraudulent or normal) from the input features
labels = data['FLAG']
input_features = data.drop(columns=['CUSTOMER', 'FLAG'])

# Replace non-numeric or null values with NaN in input features
input_features = input_features.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with zero
input_features = input_features.fillna(0)

# Stratified splitting of the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_features, labels, test_size=0.3, stratify=labels, random_state=42)

# Reshape the input data for CNN (assuming each month represents an image)
num_features = X_train.shape[1]
X_train_cnn = np.array(X_train).reshape(-1, num_features, 1)
X_test_cnn = np.array(X_test).reshape(-1, num_features, 1)

# Define the CNN model
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(num_features, 1), padding='same'))
    model.add(layers.MaxPooling1D(1))
    model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(1))
    model.add(layers.Conv1D(128, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(1))
    model.add(layers.Conv1D(256, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(1))
    model.add(layers.Conv1D(512, 3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(1))
    model.add(layers.Flatten())
    return model

# Create the CNN model
cnn_model = create_cnn_model()

# Define the LSTM model
def create_lstm_model():
    model = models.Sequential()
    model.add(layers.LSTM(128, input_shape=(num_features, 1)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Create the LSTM model
lstm_model = create_lstm_model()
time_steps = 1  # Each month considered as a single time step

# Combine the CNN and LSTM models
combined_model = models.Sequential()
combined_model.add(layers.TimeDistributed(cnn_model, input_shape=(time_steps, num_features, 1)))
combined_model.add(layers.LSTM(64))
combined_model.add(layers.Dense(1, activation='sigmoid'))

# Compile the combined model
combined_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

# Print the model summary
combined_model.summary()

# Reshape the input data for LSTM
time_steps = 1  # Each month considered as a single time step
X_train_lstm = np.array(X_train).reshape(-1, time_steps, num_features)
X_test_lstm = np.array(X_test).reshape(-1, time_steps, num_features)

# Train the combined model using the training data
history = combined_model.fit(X_train_lstm, y_train, epochs=18, batch_size=16)

# Evaluate the model on the test data
test_loss, test_accuracy = combined_model.evaluate(X_test_lstm, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)

# Plot the accuracy and loss over epochs
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()